In [1]:
from lxml import etree
import json

In [2]:
svg_file = etree.parse(open('drawing.svg', 'r')).getroot()

In [44]:
def export_markers(svg_file):
    # size of Uchiya marker in pixels
    UCHIYA_MARKER_SIZE = 600

    # version of the CelluloAR library
    VERSION = 'CelluloMarker_v1'

    # resulting array of markers
    markers = []

    # number of found markers in svg file
    n_detected = 0

    # loop over all tags
    for tag in svg_file.getchildren():
        # looking for groups
        if etree.QName(tag).localname == 'g':
            # array of circles in group
            circles = []

            # parameters of a marker
            params = None

            # subtags
            for stag in tag.getchildren():
                # name of subtag
                stag_name = etree.QName(stag).localname

                # if found a title, fill raw_name and params
                if stag_name == 'title':
                    name = stag.text.split(' ', 1)
                    if len(name) != 2 or name[0] != VERSION:
                        continue
                    params = json.loads(name[1])

                # if found a circle, fill x, y
                elif stag_name == 'circle':
                    circles.append(tuple([float(stag.get(t)) for t in ['cx', 'cy']]))

            # if the tag was a marker
            if len(circles) > 0 and params:
                # create a json object
                marker = {}

                # adding raw values to resulting json
                for key, value in params.items():
                    marker['raw_' + key] = value

                # add circles coordinates in 0-600, 0-600
                marker['dots_uchiya'] = [{"x": (x - params['x']) / params['side'] * UCHIYA_MARKER_SIZE,
                                         "y": (y - params['y']) / params['side'] * UCHIYA_MARKER_SIZE}
                                         for x, y in circles]

                # add marker name
                marker['name'] = params['id']

                # consecutive ids from 0
                marker['id'] = n_detected
                n_detected += 1

                # top-left corner
                marker['x_mm'] = params['x']
                marker['y_mm'] = params['y']

                # size of the side in mm
                marker['size_mm'] = params['side']

                # adding the marker
                markers.append(marker)
    return {'markers': markers}

In [45]:
export_markers(svg_file)

{'markers': [{'dots_uchiya': [{'x': 309.10442191269823,
     'y': 296.9876098477561},
    {'x': 495.9967283812733, 'y': 326.0683533728401},
    {'x': 139.78903912686016, 'y': 470.1393251024262},
    {'x': 534.5004243572965, 'y': 150.7382326830451},
    {'x': 238.41763373849457, 'y': 460.16599841087304},
    {'x': 539.269356900495, 'y': 352.98688614448633},
    {'x': 276.39633016088834, 'y': 236.35495560710774},
    {'x': 50.59524370530584, 'y': 197.39722326958133},
    {'x': 355.9004912862871, 'y': 86.007918118377},
    {'x': 279.0070611686209, 'y': 489.67277213282057},
    {'x': 467.8060565649839, 'y': 212.24640414656892},
    {'x': 178.130809716623, 'y': 97.13476461440378},
    {'x': 156.12200476185959, 'y': 252.5230275226097},
    {'x': 231.68522815877813, 'y': 101.66574550812805},
    {'x': 417.6253580751722, 'y': 416.2731015462499},
    {'x': 478.0386221830741, 'y': 48.52408355943738},
    {'x': 444.37043864218515, 'y': 512.3984792071966},
    {'x': 86.07603990037333, 'y': 504.159

In [ ]:
def get_w_h_from_viewbox(svg):
    x1, y1, x2, y2 = [float(x) for x in svg.get('viewBox').split()]
    return x2 - x1, y2 - y1

In [43]:
def get_w_h_mm(svg):
    vals = [svg.get(prop) for prop in ['width', 'height']]
    assert(all([val[-2:] == 'mm' for val in vals]))
    w, h = [float(val[:-2]) for val in vals]
    return w, h

In [44]:
get_w_h_from_viewbox(svg_file)

(210.0, 297.0)

In [45]:
get_w_h_mm(svg_file)

(100.0, 297.0)

In [ ]:
def 